## 기본 install

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain-community
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.5 MB/s eta 0:00:00


In [ ]:
!pip install pytesseract PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


In [ ]:
!pip install pdfplumber pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 64.3 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/STUBO/ksatparser")

In [ ]:
# Install Poppler utilities
!sudo apt-get update
!sudo apt-get install -y poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,809 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,760 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_core.documents import Document
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS
import uuid

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 20.5 MB/s eta 0:00:00


In [ ]:
import os
import base64
from PIL import Image
from IPython.display import display
from openai import OpenAI

## 성능평가

In [ ]:
def recommend_for_external_problem(target_problem, model, top_n=3):
    target_embedding = model.encode(target_problem["passage"], convert_to_tensor=False)
    target_kw_embedding = model.encode(" ".join(target_problem["keywords"]), convert_to_tensor=False)

    results = []
    for item in data:
        if item["genre"] != target_problem["genre"]:
            continue

        item_embedding = model.encode(item["passage"], convert_to_tensor=False)
        item_kw_embedding = model.encode(" ".join(item["keywords"]), convert_to_tensor=False)

        passage_sim = cosine_similarity([target_embedding], [item_embedding])[0][0]
        keyword_sim = cosine_similarity([target_kw_embedding], [item_kw_embedding])[0][0]
        final_score = 0.5 * passage_sim + 0.5 * keyword_sim

        results.append({
            "year": item["year"],
            "month": item["month"],
            "pNum": item["pNum"],
            "id_str": f"{item['year']}학년도 {item['month']} 지문 {item['pNum']}",
            "score": round(final_score, 4),
            "embedding_sim": round(passage_sim, 4),
            "keyword_cosine": round(keyword_sim, 4),
            "preview": item["passage"][:100]
        })

    return sorted(results, key=lambda x: x["score"], reverse=True)[:top_n]

In [ ]:
# STEP 1: GPT Vision으로 태깅
def tag_from_image(image_path):
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode("utf-8")

    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 국어 문제 태깅 전문가야."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text":
                     """
                          아래는 국어 영역 비문학 문제입니다.
                            이 문제에 대해 다음 정보를 태그로 생성해 주세요(최대 6개):
                            - 출제 영역(비문학)
                            - 장르('인문-언어', '인문-철학', '사회-경제', '기술-컴퓨터', '사회-법', '과학-생물', '인문-역사', '과학-화학', '인문-심리', '예술-미술', '과학-물리', '사회-정치', '예술-음악', '예술-영화' 중 택 1)
                            - 주요 주제 또는 키워드

                            태그는 쉼표로 구분해서 출력해 주세요.
                            예) 비문학, 사회-경제, 공공재, 정책 딜레마, 지방 정부 재정 지원

                            문제:
                            {text}
                     """},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        temperature=0.2
    )

    tags = response.choices[0].message.content.strip().split(",")
    tags = [t.strip() for t in tags if t.strip()]

    #print("\n[자동 태깅 결과]")
    #print("GPT 응답:", ", ".join(tags))

    return {
        "passage": "[이미지 기반 지문 생략]",
        "genre": tags[1],
        "keywords": tags[2:],
    }

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
import json

with open("/content/drive/MyDrive/STUBO/non-literature_cleaned.json", encoding="utf-8") as f:
    data = json.load(f)

with open("/content/drive/MyDrive/STUBO/evaluation_set.json", encoding="utf-8") as f:
    eval_set = json.load(f)

models = {
    "ko-sbert-nli": SentenceTransformer("jhgan/ko-sbert-nli"),
    "kr-sbert-augsts": SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS"),
    "kosimcse": SentenceTransformer("BM-K/KoSimCSE-roberta-multitask")
}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:
import unicodedata
import re

def normalize_filename(name):
    # 유니코드 정규화, 공백/특수문자/언더스코어/하이픈 제거, 소문자 변환
    name = unicodedata.normalize('NFC', name)
    name = name.replace(' ', '').replace('_', '').replace('-', '').lower()
    return name

def normalize_id(id_str):
    # '2024학년도 수능 지문 3' → '202403_3' 등으로 변환
    id_str = id_str.replace('학년도', '').replace('지문', '').replace(' ', '')
    if '수능' in id_str:
        id_str = id_str.replace('수능', '수능_')
        m = re.match(r'(\d{4})수능_(\d+)', id_str)
        if m:
            return f"{m.group(1)}03_{m.group(2)}"
    m = re.match(r'(\d{4})(\d{2})_(\d+)', id_str)
    if m:
        return f"{m.group(1)}{m.group(2)}_{m.group(3)}"
    m = re.match(r'(\d{4})_(\d{2})_(\d+)', id_str)
    if m:
        return f"{m.group(1)}{m.group(2)}_{m.group(3)}"
    return id_str


In [ ]:
# 평가셋 키 전체 정규화 (실행 전 1회)
eval_set_normalized = {normalize_filename(k): v for k, v in eval_set.items()}


In [ ]:
# ===== 4. 이미지 폴더 반복 처리 + 평가 =====
folder_path = "/content/drive/MyDrive/STUBO/유사test_기출"
image_files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

scores = {name: {"top1": 0, "top2": 0, "top3": 0} for name in models}

In [ ]:
for idx, image_name in enumerate(image_files, 1):
    image_name_norm = normalize_filename(image_name)
    image_path = os.path.join(folder_path, image_name)
    print(f"\n=== [{idx}/{len(image_files)}] 이미지: {image_name} ===")
    #gt = [normalize_id(x) for x in eval_set_normalized.get(image_name_norm, {}).get("ground_truth", [])]

    # STEP 1: GPT Vision 태깅
    external_problem = tag_from_image(image_path)

    # STEP 2: 모델별 추천 및 평가
    for model_name, model in models.items():
        print(f"\n[{model_name}] 추천 결과:")
        recommendations = recommend_for_external_problem(external_problem, model=model)

        for rank, rec in enumerate(recommendations):
            print(f"{rec['id_str']} | 점수: {rec['score']} (임베딩: {rec['embedding_sim']}, 키워드: {rec['keyword_cosine']})")
            print(f"  지문 미리보기: {rec['preview']}\n")



=== [1/20] 이미지: 2022-03-1.png ===

[ko-sbert-nli] 추천 결과:
2022학년도 03 지문 1 | 점수: 0.633899986743927 (임베딩: 0.3808000087738037, 키워드: 0.8870000243186951)
  지문 미리보기: 능숙한 독자는 어떤 능력과 태도를 지니고 있을까 ? 능숙한 
독자는 글의 의미를 이해하고 재구성하기 위해 배경지식을 효
과적으로 활용하는 능력을 지닌다 . 배경지식은 독자의 기억 

2024학년도 03 지문 1 | 점수: 0.4814000129699707 (임베딩: 0.32679998874664307, 키워드: 0.6359000205993652)
  지문 미리보기: 글에 대해 판단하고 수용하는 비판적 읽기는 글에 표면적으
로 드러난 내용이나 형식ㆍ표현을 파악하는 사실적 읽기와 , 숨
겨진 내용들을 짐작하는 추론적 읽기를 통해 글의 내용과 형


2025학년도 수능 지문 1 | 점수: 0.46480000019073486 (임베딩: 0.3116999864578247, 키워드: 0.617900013923645)
  지문 미리보기: 밑줄 긋기는 일상적으로 유용하게 활용할 수 있는 독서 전략
이다.  밑줄 긋기는 정보를 머릿속에 저장하고 기억한 내용을 
떠올리는 데 도움이 된다 .  독자로 하여금 표시한 부분에


[kr-sbert-augsts] 추천 결과:
2022학년도 03 지문 1 | 점수: 0.539900004863739 (임베딩: 0.19920000433921814, 키워드: 0.8806999921798706)
  지문 미리보기: 능숙한 독자는 어떤 능력과 태도를 지니고 있을까 ? 능숙한 
독자는 글의 의미를 이해하고 재구성하기 위해 배경지식을 효
과적으로 활용하는 능력을 지닌다 . 배경지식은 독자의 기억 

2024학년도 03 지문 1 | 점수: 0.41260001063346863 (임베딩: 0.16750000417232513, 키워드: 0.6575999855995178)

In [ ]:
# ===== 5. 정확도 통계 출력 =====
for name in scores:
    s = scores[name]
    s["total"] = s["top1"] + s["top2"] + s["top3"]

df = pd.DataFrame(scores).T
print("\n=== 모델별 Top-k 정확도 ===")
print(df[["top1", "top2", "top3", "total"]])

In [ ]:
import unicodedata

def normalize_filename(name):
    # 유니코드 정규화, 공백/특수문자 제거, 소문자 변환 등
    name = unicodedata.normalize('NFC', name)
    name = name.replace(' ', '').replace('_', '').replace('-', '').lower()
    return name

# 평가셋 키 전체 정규화
eval_set_normalized = {normalize_filename(k): v for k, v in eval_set.items()}
# 현재 파일명 정규화
image_name_normalized = normalize_filename(image_name)

# 매칭
gt = [normalize_id(x) for x in eval_set_normalized.get(image_name_normalized, {}).get("ground_truth", [])]


In [ ]:
import re

def normalize_id(id_str):
    # 예: '2024학년도 수능 지문 3' → '202403_3'
    # 예: '2022학년도 03 지문 3' → '202203_3'
    id_str = id_str.replace('학년도', '').replace('지문', '').replace(' ', '')
    # 수능은 항상 3월로 처리
    if '수능' in id_str:
        m = re.match(r'(\d{4})수능_(\d+)', id_str.replace('수능', '수능_'))
        if m:
            return f"{m.group(1)}03_{m.group(2)}"
    # 일반 월
    m = re.match(r'(\d{4})(\d{2})_(\d+)', id_str)
    if m:
        return f"{m.group(1)}{m.group(2)}_{m.group(3)}"
    # 일반 케이스: '2022_03_3'
    m = re.match(r'(\d{4})_(\d{2})_(\d+)', id_str)
    if m:
        return f"{m.group(1)}{m.group(2)}_{m.group(3)}"
    return id_str
